<a href="https://colab.research.google.com/github/Vyoma-garg/Deep_Learning/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn  #all the NN network, CNN, Modules, loss functions, activation fns
import torch.optim as optim  #SGD, ADAM
import torch.nn.functional as F  #activation fn, 
from torch.utils.data import DataLoader  #easier data managemnet, creates mini batches of the data
import torchvision.datasets as datasets   #standard datasets MNISt ,etc
import torchvision.transforms as transforms  # transformations 

Create Fully Connected Network

In [ ]:
class NN(nn.Module): #inherit from nn.Module class
    def __init__(self, input_size, num_classes):  #784,10
        super(NN,self).__init__()   #super calls the init method of the parent class which is nn.Module
        self.fc1 = nn.Linear(input_size, 50)  #784x50
        self.fc2 = nn.Linear(50, num_classes)  #50x10

    def forward(self, x):    
        x = F.relu(self.fc1(x))   
        x = self.fc2(x)
        return x



Checking 

In [5]:
#model = NN(784, 10)
#x = torch.randn(4, 784)  #randomly generated data, batch size=4, number of examples used simultaneously
#print(x)
#print('\n',model(x).shape,'\n')  #4x10   
#print(model(x)) 

tensor([[ 0.1819,  0.9657, -0.1606,  ..., -1.5237, -1.4981, -1.0362],
        [-0.2311, -0.5492, -1.6445,  ...,  1.4258, -1.1065, -0.1321],
        [ 0.5891, -0.1685,  0.6397,  ...,  0.1137, -0.7678,  0.1562],
        [ 0.7912,  1.3852,  0.0813,  ..., -1.6830,  0.9140,  2.7907]])

 torch.Size([4, 10]) 

tensor([[-0.3452,  0.0989,  0.2222,  0.0424,  0.0671, -0.4979,  0.1548, -0.0962,
          0.2268,  0.1313],
        [ 0.0110, -0.0300,  0.0645, -0.0789, -0.0730,  0.2632,  0.0038,  0.1156,
          0.0578,  0.2183],
        [ 0.0094, -0.1540, -0.0442, -0.0132, -0.1815,  0.2006,  0.2238, -0.3323,
          0.2190,  0.4252],
        [-0.1838, -0.0008,  0.0137,  0.2056, -0.1220,  0.1780,  0.0164, -0.0434,
         -0.1044,  0.4206]], grad_fn=<AddmmBackward>)


Setting device

In [6]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [7]:
input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=1

Loading dataset

In [8]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Initialize network

In [10]:
model=NN(input_size,num_classes=num_classes).to(device)

Loss and Optimizer

In [11]:
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

Train Network

In [14]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):  #enumerate helps in getting the batch index of the data 
     #Get data to cuda
      data = data.to(device)
      targets = targets.to(device)

      #print(data.shape)  #([64,1,28,28])  num of examples, number of channels(black white img), height width of each image
                        
      #get correct shape
      data=data.reshape(data.shape[0],-1)  #flatten to single dim: ([64,784])
      #print(data.shape) 

      #forward
      scores=model(data)
      loss=criterion(scores, targets) 

      #backward
      optimizer.zero_grad() #
      loss.backward()

      #gradient descent or adam step
      optimizer.step() #update weights based on the gradients



Check accuracy

In [17]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():  #while checking accuracy no need to calculate the gradients again
    for x,y in loader:
      x=x.to(device=device)
      x=x.reshape(x.shape[0],-1)

      scores=model(x)  #64x10 shape of the scores, which is max of those 10 digits
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)  #first dimension that is 64 x 10==64
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 55842/ 60000 with accuracy 93.07 

Checking Accuracy on Test Data
Got 9318/ 10000 with accuracy 93.18 

